# Llama 3 to chat about structured data
This notebook shows how to use LangChain with Llama 3 to query structured data, the 2023-24 NBA roster info, stored in a SQLite DB, and how to ask Llama 3 follow up question about the DB.

We start by installing the necessary packages:
- [Replicate](https://replicate.com/) to host the Llama 3 model
- [langchain](https://python.langchain.com/docs/get_started/introduction) provides necessary RAG tools for this demo

**Note** We will be using [Replicate](https://replicate.com/meta/meta-llama-3-8b-instruct) to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. 

If you'd like to run Llama 3 locally for the benefits of privacy, no cost or no rate limit (some Llama 3 hosting providers set limits for free plan of queries or tokens per second or minute), see [Running Llama Locally](https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Running_Llama2_Anywhere/Running_Llama_on_Mac_Windows_Linux.ipynb).

In [1]:
!pip install langchain replicate

   ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
   -------------------------------------- - 41.0/43.0 kB ? eta -:--:--
   ---------------------------------------- 43.0/43.0 kB 297.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/384.8 kB ? eta -:--:--
   ------ --------------------------------- 61.4/384.8 kB 1.7 MB/s eta 0:00:01
   ----------- ---------------------------- 112.6/384.8 kB 1.3 MB/s eta 0:00:01
   ------------------ --------------------- 174.1/384.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 276.5/384.8 kB 1.4 MB/s eta 0:00:01
   -------------------------------- ------- 317.4/384.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 384.8/384.8 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 1.0.5 requires google-generativeai<0.6.0,>=0.5.2, but you have google-generativeai 0.7.2 which is incompatible.
langchain-groq 0.1.3 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.2.30 which is incompatible.
langchain-objectbox 0.1.0 requires langchain-core<0.2.0,>=0.1.45, but you have langchain-core 0.2.30 which is incompatible.
langserve 0.1.1 requires langchain-core<0.2.0,>=0.1.0, but you have langchain-core 0.2.30 which is incompatible.


In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")

Next we call the Llama 3 8b chat model from Replicate. You can also use Llama 3 70b model by replacing the `model` name with "meta/meta-llama-3-70b-instruct".

In [4]:
from langchain_community.llms import Replicate
llm = Replicate(
    model="meta/meta-llama-3-8b-instruct",
    model_kwargs={"temperature": 0.0, "top_p": 1, "max_new_tokens":500}
)

To recreate the `nba_roster.db` file, run the two commands below:
- `python txt2csv.py` to convert the `nba.txt` file to `nba_roster.csv`. The `nba.txt` file was created by scraping the NBA roster info from the web.
- `python csv2db.py` to convert `nba_roster.csv` to `nba_roster.db`.

In [6]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info=0)

def get_schema():
    return db.get_table_info()

In [7]:
question = "What team is Klay Thompson on?"
prompt = f"""Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:
{get_schema()}

Question: {question}

SQL Query:"""

print(prompt)

Based on the table schema below, write a SQL query that would answer the user's question; just return the SQL query and nothing else.

Scheme:

CREATE TABLE nba_roster (
	"Team" TEXT, 
	"NAME" TEXT, 
	"Jersey" TEXT, 
	"POS" TEXT, 
	"AGE" INTEGER, 
	"HT" TEXT, 
	"WT" TEXT, 
	"COLLEGE" TEXT, 
	"SALARY" TEXT
)

Question: What team is Klay Thompson on?

SQL Query:


In [8]:
answer = llm.invoke(prompt)
print(answer)



SELECT "Team" FROM nba_roster WHERE "NAME" = 'Klay Thompson';


If you don't have the "just return the SQL query and nothing else" in the prompt above, or even with it but asking Llama 2 which doesn't follow instructions as well as Llama 3, you'll likely get more text other than the SQL query back in the answer.

In [9]:
# note this is a dangerous operation and for demo purpose only; in production app you'll need to safe-guard any DB operation
result = db.run(answer)

In [10]:
# how about a follow up question
follow_up = "What's his salary?"
print(llm.invoke(follow_up))



I'm happy to help! However, I need more information. Who is "his"? Could you please provide more context or clarify who you are referring to?


Since we did not pass any context along with the follow-up to the model it did not know who "his" is. Let's try to fix it by adding context to the follow-up prompt.

In [11]:
prompt = f"""Based on the table schema, question, SQL query, and SQL response below, write a new SQL response; be concise, just output the SQL response.

Scheme:
{get_schema()}

Question: {follow_up}
SQL Query: {question}
SQL Result: {result}

New SQL Response:
"""
print(prompt)

Based on the table schema, question, SQL query, and SQL response below, write a new SQL response; be concise, just output the SQL response.

Scheme:

CREATE TABLE nba_roster (
	"Team" TEXT, 
	"NAME" TEXT, 
	"Jersey" TEXT, 
	"POS" TEXT, 
	"AGE" INTEGER, 
	"HT" TEXT, 
	"WT" TEXT, 
	"COLLEGE" TEXT, 
	"SALARY" TEXT
)

Question: What's his salary?
SQL Query: What team is Klay Thompson on?
SQL Result: [('Golden State Warriors',), ('Golden State Warriors',)]

New SQL Response:



In [12]:
new_answer = llm.invoke(prompt)
print(new_answer)



SELECT SALARY FROM nba_roster WHERE "Team" = 'Golden State Warriors' AND "NAME" = 'Klay Thompson';


Because we have "be concise, just output the SQL response", Llama 3 is able to just generate the SQL statement; otherwise output parsing will be needed.

In [13]:
db.run(new_answer)

"[('$43,219,440',), ('$43,219,440',)]"